<a href="https://colab.research.google.com/github/leeds1219/DL_and_AI_Notes_and_Projects/blob/main/Generative_Adversarial_Networks_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 20. Generative Adversarial Networks

In [ ]:
# 20.1 GAN uses a two-sample test

# Generator with Noise(z) input creates a Fake G(z)
# Discriminator with input(x)
# classify if it is real or fake
# when the fake G(z) pass the classifier an image(speech etc) is generated

# The discriminator is a binary classifier to distinguish if the input(x) is real
# or fake (from generator) and gives an scalr prediction o(a real number) for input using
# a fully connected layer with hidden size 1 and then applies sigmoid function
# to obtain the predicted probability D(x) = 1/(1+exp(-o)).
# Assume the label y for the true data is 1 and fake is 0.

# We train the discriminator to minimize the cross-entropy loss
# min(D){-ylogD(x)-(1-y)log(1-D(x))}

# For the Generator it draws some parameter z from z ~ N(0,1) z is called latent variable
# it applies a function to generate x` = G(z).
# The goal of the gnerator is to fool the discriminator to classify x` = G(z) as true
# Thus we want D(G(z)) ~ 1 for given discriminator D we update the parameters of G to
# maximize cross-entropy loss when y = 0(fake)
# max(G){-(1-y)log(1-D(G(z)))} = max(G){-log(1-D(G(z)))}

# If D(x`) ~ 1 so loss is near 0.
# So we minimize the following loss
# min(G){-ylog(D(G(z)))} = min(G){-log(D(G(z)))}

# which is just feeding x` = G(z) into the discriminator but giving label y = 0(true)
# In other words, D and G are playing a "minimax" game!!!
# min(D)max(G){-E_x~DatalogD(x)-E_z~Noiselog(1-D(G(z)))}

#  We will illustrate what happens if we use GANs to build
# the world’s most inefficient estimator of parameters for a Gaussian

In [ ]:
%matplotlib inline
import torch
from torch import nn
from d2l import torch as d2l